In [1]:
from neo4j import GraphDatabase

uri = 'neo4j://localhost:7687'
user = 'neo4j'
password = 'password'

driver = GraphDatabase.driver(uri, auth=(user, password))

Домашнее задание 30 (необходимо сделать 3 задачи из 4) на Python:
1. Найти самый старый фильм и самый новый фильм - вывести их названия по 1 шт (2 запроса)

In [6]:
def get_oldest_movie(tx):
    result = tx.run("""
            MATCH (m:Movie)
            RETURN m.title as oldest_movie, m.released
            ORDER BY m.released ASC
            LIMIT 1
    """
    )
    return [{record['oldest_movie']: record["m.released"]} for record in result]



def get_newest_movie(tx):
    result = tx.run("""
            MATCH (m:Movie)
            RETURN m.title as newest_movie, m.released
            ORDER BY m.released DESC
            LIMIT 1
    """
    )
    return [{record['newest_movie']: record["m.released"]} for record in result]


with driver.session() as session:
    oldest_movie = session.execute_read(get_oldest_movie)
    print(oldest_movie)

with driver.session() as session:
    newest_movie = session.execute_read(get_newest_movie)
    print(newest_movie)

[{"One Flew Over the Cuckoo's Nest": 1975}]
[{'Cloud Atlas': 2012}]


2. Получить среднее количество актёров на фильм 

In [8]:
def get_avg_actors_per_movie(tx):
    result = tx.run("""
            MATCH (m:Movie)<-[:ACTED_IN]-(actor:Person)
            WITH m, COUNT(actor) as actors_count
            RETURN AVG(actors_count) as avg_actors_per_movie
    """
    )
    return result.single()["avg_actors_per_movie"]


with driver.session() as session:
    avg_actors_per_movie = session.execute_read(get_avg_actors_per_movie)
    print(avg_actors_per_movie)

4.526315789473684


3. Группировка фильмов по годам и подсчёт количества фильмов в каждом году

In [9]:
def get_movie_count_per_year(tx):
    result = tx.run("""
            MATCH (m:Movie)
            WITH m.released as year_released, COUNT(m) as movie_count
            RETURN year_released, movie_count
            ORDER BY year_released
    """
    )
    return [{record['year_released']: record["movie_count"]} for record in result]


with driver.session() as session:
    movie_count_per_year = session.execute_read(get_movie_count_per_year)
    print(movie_count_per_year)

[{1975: 1}, {1986: 2}, {1990: 1}, {1992: 4}, {1993: 1}, {1995: 2}, {1996: 3}, {1997: 2}, {1998: 3}, {1999: 4}, {2000: 3}, {2003: 3}, {2004: 1}, {2006: 3}, {2007: 1}, {2008: 2}, {2009: 1}, {2012: 1}]


4. Найти актёра с наибольшим промежутком между первым и последним фильмом

In [10]:
def get_actor_with_max_career_span(tx):
    result = tx.run("""
            MATCH (actor:Person)-[:ACTED_IN]->(m:Movie)
            WITH actor, MAX(m.released) as last_movie_year, MIN(m.released) as first_movie_year
            RETURN actor.name as actor, (last_movie_year - first_movie_year) as career_span
            ORDER BY career_span DESC
            LIMIT 1
    """
    )
    return [{record['actor']: record["career_span"]} for record in result]


with driver.session() as session:
    actor_with_max_career_span = session.execute_read(get_actor_with_max_career_span)
    print(actor_with_max_career_span)

[{'Jack Nicholson': 28}]
